**Understand Cache in spark:**
- referenct to undrestand cache in spark: https://luminousmen.com/post/explaining-the-mechanics-of-spark-caching/
- cache is the process of materialize rdd in ram or disk,
- two method to perform cache: cache(), persist(persist_level)
- cache: will store rdd in ram if possible, if not store in disk
- persist: store rdd base on specify persist level
- cache is not action and is not materilize until an action is call.

In [ ]:
#example of using cache() and unpersist().
df = spark.read.parquet(data_path)
df.cache() # Cache the data
df.count() # Materialize the cache, command took 5.11 seconds
df.is_cached # Determining whether a dataframe is cached
df.count() # Now get it from the cache, command took 0.44 seconds
df.storageLevel # Determing the persistent type (memory, deserialized, # replicas)
df.unpersist()

In [ ]:
#example of using cache with tempview and spark sql api
df.createOrReplaceTempView('df')
spark.catalog.cacheTable('df')
spark.catalog.isCached(tableName='df')
spark.catalog.uncacheTable('df')
spark.catalog.clearCache() # Clear all the tables from cache

**Understand cache sequence in spark:**
- spark command will go through logical planning and physical planning
- logical planning: analyzer -> cache manager -> optimizer. cache manager will decide cache after analyzer. Therefore, cache will not use if we have different analyzer plan

In [ ]:
#example of cache is not use because we have different analyzer plan
#it is best pratice to cache dataframe which we want to do many action on it.
df = spark.read.parquet(data_path)
df.select(col1, col2).filter(col2 > 0).cache()
df.filter(col2 > 0).select(col1, col2)